In [1]:
import pennylane as qml
from pennylane import numpy as np

# Settings
n_qubits = 6  # High-dimensional space
reduced_qubits = 2  # Target dimensionality
dev = qml.device("default.qubit", wires=n_qubits)

# Example: random classical data vector (normalized)
x = np.random.rand(2**n_qubits)
x = x / np.linalg.norm(x)

@qml.qnode(dev)
def qrdr_circuit(params):
    # Step 1: Encode x using MottonenStatePreparation
    qml.MottonenStatePreparation(x, wires=range(n_qubits))
    
    # Step 2: Apply parametrized layers to "highlight resonances"
    for i in range(n_qubits):
        qml.RY(params[i], wires=i)
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i+1])

    # Step 3: Measure reduced observable (e.g., project to a 2D subspace)
    return [qml.expval(qml.PauliZ(i)) for i in range(reduced_qubits)]

# Random initialization
params = np.random.uniform(0, 2 * np.pi, n_qubits)

# Output: compressed representation
compressed = qrdr_circuit(params)
print("Compressed representation:", compressed)


Compressed representation: [tensor(-0.61696429, requires_grad=True), tensor(-0.80184297, requires_grad=True)]


In [5]:
import numpy as np
from scipy.linalg import expm, eigh

# === Utility functions ===

def kron(*args):
    """Left-to-right Kronecker product."""
    result = args[0]
    for a in args[1:]:
        result = np.kron(result, a)
    return result

def normalize(v):
    """Normalize a vector to unit norm."""
    return v / np.linalg.norm(v)


# === Core QRDR Function ===

def qrdr_single_sample(x, A, R=2, c=0.01):
    """
    Simulate QRDR for a single input vector x using covariance matrix A.
    
    Args:
        x (np.ndarray): Input vector (length N)
        A (np.ndarray): Covariance matrix (N x N)
        R (int): Reduced dimension
        c (float): Resonant parameter

    Returns:
        dict with keys: psi_out, psi_post, reduced_state, z_projected, success_prob
    """
    N = len(x)
    t = 1 / c

    # Step 1: Diagonalize covariance matrix
    evals, evecs = eigh(A)
    idx = np.argsort(evals)[::-1]
    evals = evals[idx]
    evecs = evecs[:, idx]
    V = evecs[:, :R]      # Top R eigenvectors
    lambdas = evals[:R]   # Top R eigenvalues

    # Step 2: Build Hamiltonian
    I_r = np.eye(R)
    I_n = np.eye(N)
    sigma_y = np.array([[0, -1j], [1j, 0]])

    # B matrix (approximate Hadamard-like interaction)
    B = np.zeros((R, R))
    for i in range(R):
        for j in range(R):
            B[i, j] = 1 / np.sqrt(R) * (-1)**(bin(i & j).count("1"))

    # H_lambda for eigenvalues
    H_lambda = -np.diag(lambdas)

    # Hp = H_lambda ⊗ I_n + I_r ⊗ A
    Hp = kron(H_lambda, I_n) + kron(I_r, A)

    # Full Hamiltonian H
    H_probe0 = kron(np.array([[1, 0], [0, 0]]), kron(np.zeros((R, R)), I_n))  # irrelevant, can be zero
    H_probe1 = kron(np.array([[0, 0], [0, 1]]), Hp)
    interaction = (c * np.pi / 2) * kron(sigma_y, kron(B, I_n))
    H = H_probe0 + H_probe1 + interaction

    # Step 3: Build input state |0⟩⊗|0⟩⊗|x⟩
    psi = np.zeros((2 * R * N,), dtype=complex)
    psi_idx = 0 * (R * N) + 0 * N + np.arange(N)
    psi[psi_idx] = normalize(x)

    # Step 4: Time evolution
    U = expm(-1j * H * t)
    psi_out = U @ psi

    # Step 5: Postselect on probe = |1⟩
    proj1_idx = 1 * (R * N) + np.arange(R * N)
    proj1 = np.zeros_like(psi_out)
    proj1[proj1_idx] = psi_out[proj1_idx]

    norm = np.linalg.norm(proj1)
    if norm < 1e-10:
        print("Postselection failed.")
        return None

    psi_post = proj1 / norm
    psi_matrix = psi_post.reshape(2, R, N)[1]  # Extract |1⟩ part

    # Project onto eigenbasis
    z = psi_matrix @ V  # [R x N] @ [N x R] = [R x R]

    return {
        "psi_out": psi_out,
        "psi_post": psi_post,
        "reduced_state": psi_matrix,
        "z_projected": z,
        "success_prob": norm**2,
    }


# === Example Usage ===

# Example setup
N = 4
np.random.seed(42)
x = normalize(np.random.randn(N))
A = np.outer(x, x) + 0.01 * np.eye(N)  # covariance matrix with regularization

result = qrdr_single_sample(x, A, R=2, c=0.01)

# Display results
print("Postselection Success Probability:", result["success_prob"])
print("Projected Components z:\n", result["z_projected"])


Postselection Success Probability: 0.5000280411145741
Projected Components z:
 [[ 9.99971960e-01-1.91798726e-15j -3.31634560e-16+2.20194547e-17j]
 [-7.22624165e-03-1.96481953e-03j -3.56601985e-16-6.65392293e-18j]]
